In [1]:
import tensorflow as tf
import numpy as np
from time import time
import os
print(tf.__version__)

2022-05-02 15:33:14.481073: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


2.3.0


### Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")
    
    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split("-")[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

### Data load & pre-processing function

In [3]:
def load_mnist():
    (train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.mnist.load_data()

    train_data = np.expand_dims(train_data, axis=-1)  # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1)  # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)
    
    train_labels = tf.keras.utils.to_categorical(train_labels, 10)  # [N, ] -> [N, 10]
    test_labels = tf.keras.utils.to_categorical(test_labels, 10)  # [N, ] -> [N, 10]
    
    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

### Performance function

In [4]:
def loss_function(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits,
                                                                  y_true=labels,
                                                                  from_logits=True))
    return loss

def accuracy_function(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_function(model, images, labels)
    return tape.gradient(loss, model.variables)

### Create model (class version)

In [5]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()
        
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Flatten())
        
        for i in range(4):
            self.model.add(tf.keras.layers.Dense(units=512, use_bias=True, kernel_initializer=weight_init))
            self.model.add(tf.keras.layers.Activation(tf.keras.activations.relu))
            self.model.add(tf.keras.layers.Dropout(0.5))
        
        self.model.add(tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init))
    
    def call(self, x, training=None, mask=None):
        x = self.model(x)
        return x

### Define data & hyper-parameter

In [6]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
        shuffle(buffer_size=100000).\
        prefetch(buffer_size=batch_size).\
        batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
        shuffle(buffer_size=100000).\
        prefetch(buffer_size=len(test_x)).\
        batch(len(test_x))

2022-05-02 15:33:16.153384: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-05-02 15:33:16.221660: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.71GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.23GiB/s
2022-05-02 15:33:16.222266: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:04:00.0 name: GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.71GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.23GiB/s
2022-05-02 15:33:16.222289: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-05-02 15:33:16.223942: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-05

### Define model & optimizer & writer

In [9]:
""" Model """
network = create_model_class(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = "checkpoints"
logs_dir = "logs"

model_dir = "nn_dropout"

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

### Restore checkpoint & start train or test phase

In [11]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint_counter
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))
                
                train_loss = loss_function(network, train_input, train_label)
                train_accuracy = accuracy_function(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_function(network, test_input, test_label)
                
                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + "-{}".format(counter))

# test phase
else:
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:
        test_accuracy = accuracy_function(network, test_input, test_label)
    
    print("test_Accuracy : %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.8449, train_loss: 2.34645748, train_accuracy: 0.2578, test_Accuracy: 0.1739
Epoch: [ 0] [    1/  468] time: 1.1099, train_loss: 2.27017355, train_accuracy: 0.2500, test_Accuracy: 0.2124
Epoch: [ 0] [    2/  468] time: 1.4030, train_loss: 2.36953688, train_accuracy: 0.3828, test_Accuracy: 0.3213
Epoch: [ 0] [    3/  468] time: 1.6560, train_loss: 2.31015110, train_accuracy: 0.3672, test_Accuracy: 0.3429
Epoch: [ 0] [    4/  468] time: 1.9236, train_loss: 2.26975107, train_accuracy: 0.3672, test_Accuracy: 0.3920
Epoch: [ 0] [    5/  468] time: 2.1710, train_loss: 2.29302001, train_accuracy: 0.4219, test_Accuracy: 0.4577
Epoch: [ 0] [    6/  468] time: 2.4318, train_loss: 2.30089092, train_accuracy: 0.5391, test_Accuracy: 0.5309
Epoch: [ 0] [    7/  468] time: 2.6785, train_loss: 2.19896269, train_accuracy: 0.6250, test_Accuracy: 0.5707
Epoch: [ 0] [    8/  468] time: 2.9334,

Epoch: [ 0] [   74/  468] time: 19.8357, train_loss: 0.48446676, train_accuracy: 0.9297, test_Accuracy: 0.8876
Epoch: [ 0] [   75/  468] time: 20.0949, train_loss: 0.50025535, train_accuracy: 0.9062, test_Accuracy: 0.8882
Epoch: [ 0] [   76/  468] time: 20.3350, train_loss: 0.50197393, train_accuracy: 0.9062, test_Accuracy: 0.8894
Epoch: [ 0] [   77/  468] time: 20.5770, train_loss: 0.57157552, train_accuracy: 0.8594, test_Accuracy: 0.8907
Epoch: [ 0] [   78/  468] time: 20.8159, train_loss: 0.56329012, train_accuracy: 0.8750, test_Accuracy: 0.8901
Epoch: [ 0] [   79/  468] time: 21.0890, train_loss: 0.45902640, train_accuracy: 0.8906, test_Accuracy: 0.8905
Epoch: [ 0] [   80/  468] time: 21.3379, train_loss: 0.52333134, train_accuracy: 0.8984, test_Accuracy: 0.8901
Epoch: [ 0] [   81/  468] time: 21.5976, train_loss: 0.48881951, train_accuracy: 0.9062, test_Accuracy: 0.8910
Epoch: [ 0] [   82/  468] time: 21.8520, train_loss: 0.45544076, train_accuracy: 0.9375, test_Accuracy: 0.8929
E

Epoch: [ 0] [  148/  468] time: 38.6825, train_loss: 0.39020938, train_accuracy: 0.9062, test_Accuracy: 0.9257
Epoch: [ 0] [  149/  468] time: 38.9377, train_loss: 0.40506798, train_accuracy: 0.9531, test_Accuracy: 0.9255
Epoch: [ 0] [  150/  468] time: 39.2018, train_loss: 0.31283879, train_accuracy: 0.9375, test_Accuracy: 0.9249
Epoch: [ 0] [  151/  468] time: 39.4562, train_loss: 0.38056269, train_accuracy: 0.9297, test_Accuracy: 0.9258
Epoch: [ 0] [  152/  468] time: 39.7223, train_loss: 0.38553569, train_accuracy: 0.9219, test_Accuracy: 0.9256
Epoch: [ 0] [  153/  468] time: 39.9746, train_loss: 0.36402532, train_accuracy: 0.9219, test_Accuracy: 0.9232
Epoch: [ 0] [  154/  468] time: 40.2417, train_loss: 0.50257123, train_accuracy: 0.8594, test_Accuracy: 0.9235
Epoch: [ 0] [  155/  468] time: 40.4942, train_loss: 0.25666091, train_accuracy: 0.9375, test_Accuracy: 0.9237
Epoch: [ 0] [  156/  468] time: 40.7486, train_loss: 0.52278292, train_accuracy: 0.9062, test_Accuracy: 0.9230
E

Epoch: [ 0] [  222/  468] time: 57.5133, train_loss: 0.30184394, train_accuracy: 0.9453, test_Accuracy: 0.9365
Epoch: [ 0] [  223/  468] time: 57.7698, train_loss: 0.42085215, train_accuracy: 0.9375, test_Accuracy: 0.9367
Epoch: [ 0] [  224/  468] time: 58.0284, train_loss: 0.35594785, train_accuracy: 0.9531, test_Accuracy: 0.9364
Epoch: [ 0] [  225/  468] time: 58.2781, train_loss: 0.32087469, train_accuracy: 0.8984, test_Accuracy: 0.9379
Epoch: [ 0] [  226/  468] time: 58.5304, train_loss: 0.35712880, train_accuracy: 0.9141, test_Accuracy: 0.9378
Epoch: [ 0] [  227/  468] time: 58.7937, train_loss: 0.30084747, train_accuracy: 0.9688, test_Accuracy: 0.9371
Epoch: [ 0] [  228/  468] time: 59.0465, train_loss: 0.38376063, train_accuracy: 0.9219, test_Accuracy: 0.9373
Epoch: [ 0] [  229/  468] time: 59.3151, train_loss: 0.41347772, train_accuracy: 0.9453, test_Accuracy: 0.9358
Epoch: [ 0] [  230/  468] time: 59.5533, train_loss: 0.29843152, train_accuracy: 0.9453, test_Accuracy: 0.9351
E

Epoch: [ 0] [  296/  468] time: 76.8260, train_loss: 0.16351086, train_accuracy: 0.9922, test_Accuracy: 0.9452
Epoch: [ 0] [  297/  468] time: 77.0960, train_loss: 0.22146192, train_accuracy: 0.9453, test_Accuracy: 0.9455
Epoch: [ 0] [  298/  468] time: 77.3688, train_loss: 0.20153818, train_accuracy: 0.9688, test_Accuracy: 0.9451
Epoch: [ 0] [  299/  468] time: 77.6305, train_loss: 0.21956776, train_accuracy: 0.9531, test_Accuracy: 0.9453
Epoch: [ 0] [  300/  468] time: 77.8990, train_loss: 0.25598294, train_accuracy: 0.9531, test_Accuracy: 0.9462
Epoch: [ 0] [  301/  468] time: 78.1721, train_loss: 0.29738590, train_accuracy: 0.9375, test_Accuracy: 0.9479
Epoch: [ 0] [  302/  468] time: 78.4454, train_loss: 0.33566719, train_accuracy: 0.9531, test_Accuracy: 0.9476
Epoch: [ 0] [  303/  468] time: 78.7090, train_loss: 0.33022967, train_accuracy: 0.9297, test_Accuracy: 0.9487
Epoch: [ 0] [  304/  468] time: 78.9761, train_loss: 0.34560940, train_accuracy: 0.9688, test_Accuracy: 0.9492
E

Epoch: [ 0] [  370/  468] time: 96.0396, train_loss: 0.23612240, train_accuracy: 0.9375, test_Accuracy: 0.9516
Epoch: [ 0] [  371/  468] time: 96.3063, train_loss: 0.33279687, train_accuracy: 0.9219, test_Accuracy: 0.9523
Epoch: [ 0] [  372/  468] time: 96.5772, train_loss: 0.28549233, train_accuracy: 0.9609, test_Accuracy: 0.9527
Epoch: [ 0] [  373/  468] time: 96.8292, train_loss: 0.23923597, train_accuracy: 0.9531, test_Accuracy: 0.9526
Epoch: [ 0] [  374/  468] time: 97.0774, train_loss: 0.13133813, train_accuracy: 0.9688, test_Accuracy: 0.9520
Epoch: [ 0] [  375/  468] time: 97.3237, train_loss: 0.22282940, train_accuracy: 0.9531, test_Accuracy: 0.9518
Epoch: [ 0] [  376/  468] time: 97.5850, train_loss: 0.24237990, train_accuracy: 0.9375, test_Accuracy: 0.9525
Epoch: [ 0] [  377/  468] time: 97.8463, train_loss: 0.28381047, train_accuracy: 0.9609, test_Accuracy: 0.9530
Epoch: [ 0] [  378/  468] time: 98.1062, train_loss: 0.27189592, train_accuracy: 0.9531, test_Accuracy: 0.9540
E

Epoch: [ 0] [  444/  468] time: 115.7884, train_loss: 0.19348082, train_accuracy: 0.9609, test_Accuracy: 0.9572
Epoch: [ 0] [  445/  468] time: 116.0330, train_loss: 0.18586010, train_accuracy: 0.9609, test_Accuracy: 0.9572
Epoch: [ 0] [  446/  468] time: 116.2885, train_loss: 0.18921685, train_accuracy: 0.9766, test_Accuracy: 0.9576
Epoch: [ 0] [  447/  468] time: 116.5243, train_loss: 0.25227141, train_accuracy: 0.9688, test_Accuracy: 0.9574
Epoch: [ 0] [  448/  468] time: 116.7542, train_loss: 0.15675905, train_accuracy: 0.9844, test_Accuracy: 0.9568
Epoch: [ 0] [  449/  468] time: 116.9859, train_loss: 0.16592905, train_accuracy: 0.9688, test_Accuracy: 0.9568
Epoch: [ 0] [  450/  468] time: 117.2188, train_loss: 0.30192894, train_accuracy: 0.9609, test_Accuracy: 0.9562
Epoch: [ 0] [  451/  468] time: 117.4680, train_loss: 0.16796684, train_accuracy: 0.9844, test_Accuracy: 0.9566
Epoch: [ 0] [  452/  468] time: 117.7268, train_loss: 0.24408396, train_accuracy: 0.9453, test_Accuracy: